In [8]:

from pydantic_ai import Agent, RunContext
from pydantic_ai.common_tools.tavily import tavily_search_tool
from pydantic_ai.messages import ModelMessage
from pydantic_ai.models.google import GoogleModel
from pydantic_ai.providers.google import GoogleProvider
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
from dotenv import load_dotenv
from dataclasses import dataclass
from datetime import datetime
from pydantic import Field


import nest_asyncio
nest_asyncio.apply()


import os
from tavily import TavilyClient
load_dotenv()
google_api_key=os.getenv('google_api_key')
tavily_key=os.getenv('tavily_key')
pse=os.getenv('pse')
openai_api_key=os.getenv('openai_api_key')
tavily_client = TavilyClient(api_key=tavily_key)
composio_api_key=os.getenv('composio_api_key')
# configure logfire
# import logfire
# logfire.configure(token=os.getenv('logfire_token'))
# logfire.instrument_pydantic_ai()


In [9]:

api_keys={
    'google_api_key':google_api_key,
    'tavily_key':tavily_key,
    'pse':pse,
    'openai_api_key':openai_api_key,
    'composio_key':composio_api_key
}


In [10]:
from src.cortana.cortana_agent import Cortana_agent
from src.cortana.PrebuiltTools.google_tools import search_images_tool, code_execution_tool
from src.cortana.utils.helper_functions import MCP_server_helper

In [11]:
mcp_server_helper=MCP_server_helper()
mcp_server_helper.add_mpc_server(type='http', mpc_server_url=os.getenv('gmail_mcp_server'), headers=None)
servers=mcp_server_helper.get_mpc_servers()

In [12]:
tools=[search_images_tool(api_key=api_keys['google_api_key'],search_engine_id=api_keys['pse']),code_execution_tool(api_key=api_keys['google_api_key'])]

In [13]:
llm=OpenAIModel('gpt-4.1-mini',provider=OpenAIProvider(api_key=api_keys['openai_api_key']))

In [14]:
agent=Cortana_agent(llm=llm, tools=tools, mpc_servers=servers, summarizer=True)

In [15]:
result=await agent.chat(['what is 10+10*2.5'])
print(result.ui_version)

Connected to MCP server
The result of the expression 10 + 10 * 2.5 is 35.0.


In [ ]:

llms={'pydantic_llm':GoogleModel('gemini-2.5-flash', provider=GoogleProvider(api_key=api_keys['google_api_key']))}